## 最小二乘法和脊回归

In [1]:
import xlrd
from sklearn import linear_model
#import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def get_error2(tranData, testData, name, func,evaluation):  
#tranData这个数据集是训练集
#testData是测试集
#name是数据集中变量名
#func为回归函数
#evaluation为误差判断函数

    error = {}

#    print name
    Y = tranData['PE']
    Y1 =testData['PE']
    for i in name:

            X = tranData[i]
            X1 = testData[i]
            clf = func()
            clf.fit( X ,Y)
            print(i, clf.coef_)
            print(i, clf.intercept_)
            error[i+"训练误差"]=evaluation(Y,clf.predict(X))
            print(i+"训练误差:", str(error[i+"训练误差"]))
            error[i+"测试误差"]= evaluation(Y1, clf.predict(X1))
            print(i+"测试误差:", error[i+"测试误差"])

    return error

def getData():
    data = xlrd.open_workbook('./CCPP/Folds5x2_pp.xlsx')

    #读取数据，将xls中的数据读取出来
    sheet1 = data.sheet_by_name('Sheet1')
    sheet2 = data.sheet_by_name('Sheet2')
    sheet3 = data.sheet_by_name('Sheet3')
    sheet4 = data.sheet_by_name('Sheet4')
    sheet5 = data.sheet_by_name('Sheet5')

    #训练集
    AT_tran = sheet1.col_values(0)[1:] + sheet2.col_values(0)[1:] + sheet3.col_values(0)[1:] 
    V_tran = sheet1.col_values(1)[1:] + sheet2.col_values(1)[1:] + sheet3.col_values(1)[1:]
    AP_tran = sheet1.col_values(2)[1:] + sheet2.col_values(2)[1:] + sheet3.col_values(2) [1:]
    RH_tran = sheet1.col_values(3)[1:] + sheet2.col_values(3)[1:] + sheet3.col_values(3)[1:]  
    PE_tran = sheet1.col_values(4)[1:] + sheet2.col_values(4)[1:] + sheet3.col_values(4)[1:]

    #测试集
    AT_test = sheet4.col_values(0)[1:] + sheet5.col_values(0)[1:]
    V_test = sheet4.col_values(1)[1:] + sheet5.col_values(1)[1:]
    AP_test =  sheet4.col_values(2)[1:] + sheet5.col_values(2)[1:]
    RH_test = sheet4.col_values(3)[1:] + sheet5.col_values(3)[1:]
    PE_test = sheet4.col_values(4)[1:] +  sheet5.col_values(4)[1:]

    tran = {'AT':AT_tran,'V':V_tran,'AP':AP_tran,'RH':RH_tran,'PE':PE_tran}
    test = {'AT':AT_test,'V':V_test,'AP':AP_test,'RH':RH_test,'PE':PE_test}

    return tran, test


#拟合函数
def getResultError2(tran, test, regFunc, errFunc):
#tran训练集
#test测试集
#regFunc回归函数
#errFunc误差测量函数
#parameter参数
    name1 = ['AT', 'V', 'AP', 'RH']
    tran1 = {'AT':[[x] for x in tran['AT']], 'V':[[x] for x in tran['V']], 'AP':[[x] for x in tran['AP']],'RH':[[x] for x in tran['RH']], 'PE':[[x] for x in tran['PE']]}
    test1 = {'AT':[[x] for x in test['AT']], 'V':[[x] for x in test['V']], 'AP':[[x] for x in test['AP']],'RH':[[x] for x in test['RH']], 'PE':[[x] for x in test['PE']]}
    error1 = get_error2(tran1,  test1, name1, regFunc, errFunc)

    #数据子集有两个属性
    tran2 = {'AT-V':[[x, y] for x, y in zip(tran['AT'], tran['V'])],'AT-AP':[[x, y] for x, y in zip(tran['AT'], tran['AP'])],  'AT-RH':[[x, y] for x, y in zip(tran['AT'], tran['RH'])], 'V-AP':[[x, y] for x, y in zip(tran['V'], tran['AP'])], 'V-RH':[[x, y] for x, y in zip(tran['V'], tran['AP'])] ,'PE':[[x] for x in tran['PE']]}
    test2 = {'AT-V': [[x, y] for x, y in zip(test['AT'],test['V'])],'AT-AP':[[x, y] for x, y in  zip(test['AT'], test['AP'])],  'AT-RH': [[x, y] for x, y in zip(test['AT'], test['RH'])], 'V-AP':[[x, y ] for x, y in zip(test['V'], test['AP'])], 'V-RH':[[x, y] for x, y in zip(test['V'], test['AP'])],'PE': [[x] for x in test['PE']]}

    name2 = [ 'AT-V', 'AT-AP', 'AT-RH', 'V-AP', 'V-RH']
    error2 = get_error2(tran2,  test2, name2, regFunc, errFunc)


    #数据子集有三个属性
    tran3 = {'AT-V-AP':[[x, y, z] for x, y, z in zip(tran['AT'], tran['V'] , tran['AP'])],'AT-V-RH':[[x, y, z] for x, y, z in zip(tran['AT'], tran['V'], tran['RH'])],  'V-AP-RH':[[x, y, z] for x, y, z in zip(tran['V'], tran['AP'], tran['RH'])], 'PE':[[x] for x in tran['PE']]}
    test3 = {'AT-V-AP':[[x, y, z] for x, y, z in zip(test['AT'],test['V'] , test['AP'])],'AT-V-RH':[[x, y, z] for x, y, z in zip(test['AT'],test['V'], test['RH'])],  'V-AP-RH':[[x, y, z] for x, y, z in zip(test['V'], test['AP'], test['RH'])], 'PE':[[x] for x in test['PE']]}
    name3 = [ 'AT-V-AP', 'AT-V-RH', 'V-AP-RH' ]
    error3 = get_error2(tran3,  test3, name3, regFunc, errFunc)

    #数据子集有四个属性
    tran4 = {'AT-V-AP-RH':[[x, y, z, w] for x, y, z, w in zip(tran['AT'], tran['V'] , tran['AP'], tran['RH'])], 'PE':[[x] for x in tran['PE']]}
    test4 = {'AT-V-AP-RH':[[x, y, z, w] for x, y, z, w in zip(tran['AT'],test['V'] , test['AP'], test['RH'])], 'PE':[[x] for x in test['PE']]}
    name4 = [ 'AT-V-AP-RH' ]
    error4 = get_error2(tran4,  test4, name4, regFunc, errFunc)

    finalError = {1:error1, 2:error2, 3:error3, 4:error4}
    return finalError

if __name__ == '__main__':

    tran, test = getData()

    print("================最小二乘发回归=================")
    print("××××××××××××××××平均绝对偏差*****************")
    linearAbsoluteError = getResultError2(tran, test, linear_model.LinearRegression , mean_absolute_error)
    #linear_model.LinearRegression为scikit-learn中的最小二乘法中的函数，mean_absolute_error绝对误差衡量函数。在python中，函数也是可以传入函数中的。
    print("******************均方差********************")
    linearSquareError = getResultError2(tran, test, linear_model.LinearRegression , mean_squared_error)
    #mean_squared_error标准差衡量函数
    print("================最小二乘发回归=================")

    print("================脊回归=================")
    print("××××××××××××××××平均绝对偏差*****************")
    linearAbsoluteError = getResultError2(tran, test, linear_model.Ridge , mean_absolute_error)
    print ("******************均方差********************")
    linearSquareError = getResultError2(tran, test, linear_model.Ridge , mean_squared_error)
    print("================脊回归=================")

================最小二乘发回归=================
××××××××××××××××平均绝对偏差*****************
AT [[-2.17131996]]
AT [ 497.03411989]
AT训练误差: 4.2879818941
AT测试误差: 4.2879818941
V [[-1.16813513]]
V [ 517.80152631]
V训练误差: 6.57649440109
V测试误差: 6.57649440109
AP [[ 1.48987167]]
AP [-1055.26098898]
AP训练误差: 11.9575739793
AP测试误差: 11.9575739793
RH [[ 0.4556501]]
RH [ 420.96176616]
RH训练误差: 13.1940215836
RH测试误差: 13.1940215836
AT-V [[-1.70426591 -0.32448652]]
AT-V [ 505.47743382]
AT-V训练误差: 3.91742599695
AT-V测试误差: 3.91742599695
AT-AP [[-2.11306594  0.14402919]]
AT-AP [ 349.95047262]
AT-AP训练误差: 4.27818773199
AT-AP测试误差: 4.27818773199
AT-RH [[-2.39070329 -0.20640286]]
AT-RH [ 516.47645545]
AT-RH训练误差: 3.79779100865
AT-RH测试误差: 3.79779100865
V-AP [[-1.06177593  0.55039349]]
V-AP [-45.66559505]
V-AP训练误差: 6.1510549951
V-AP测试误差: 6.1510549951
V-RH [[-1.06177593  0.55039349]]
V-RH [-45.66559505]
V-RH训练误差: 6.1510549951
V-RH测试误差: 6.1510549951
AT-V-AP [[-1.63477738 -0.32832324  0.15815192]]
AT-V-AP [ 344.0713874]
AT-V-AP训练误差: 3